### For Docker container
run this command to access its terminal
```bash
docker exec -it ai-mock-coding-interview-agent-fastapi-1 /bin/bash
```

In [ ]:
from agents.main_graph import main_graph
id = "6734f462996d5ea7277bd903"
config = {"configurable": {"thread_id": id}, "recursion_limit": 100}
state = main_graph.get_state(config).values
state

In [5]:
def code_to_single_line(code_string: str) -> str:
    # Replace tabs/spaces with \t and newlines with \n
    lines = code_string.split("\n")
    # Process each line to convert leading spaces/tabs to \t
    processed_lines = []
    for line in lines:
        # Count leading spaces (assuming 4 spaces = 1 tab)
        leading_spaces = len(line) - len(line.lstrip())
        tab_count = leading_spaces // 4
        # Replace the leading spaces with \t characters
        processed_line = "\t" * tab_count + line.lstrip()
        processed_lines.append(processed_line)

    # Join with literal \n
    return "\\n".join(processed_lines)

In [21]:
import json
from markdownify import markdownify

path = "/Users/minkijung/Documents/2PetProjects/ai-mock-coding-interview-agent/backend/db/leetcode/scraped_data/longest-common-prefix.json"

with open(path, "r") as f:
    data = json.load(f)

solution_md = markdownify(data["solution"]["content"])
content_md = markdownify(data["content"])

data["solution_md"] = solution_md
data["content_md"] = content_md

with open(path, "w") as f:
    json.dump(data, f, indent=4)

1. Create a function that uses the first element of the list as a reference.
2. Iteratively check if the first n characters of the reference are included in all other elements.
3. Increase n until finding the longest common prefix.
4. Add an assertion to check for empty elements in the array.
5. Consider handling special characters as an edge case.
6. Implement the solution and analyze time complexity during coding.

In [23]:
from pydantic import BaseModel, Field
class DebuggingAgentPrivateState(BaseModel):
    interview_question: str = Field(default="")
    interview_solution: str = Field(default="")
    code_editor_state: str = Field(default="")

    debugging_record: list[str] = Field(default=[])

state = DebuggingAgentPrivateState()
state.__annotations__

{'interview_question': str,
 'interview_solution': str,
 'code_editor_state': str,
 'debugging_record': list[str]}

# populate prep_code

In [6]:
import json 
with open(
    "/Users/minkijung/Documents/2PetProjects/ai-mock-coding-interview-agent/backend/db/leetcode.json",
    "r",
) as f:
    allquestions = json.load(f)

id_title_dict = {}
for question in allquestions:
    id_title_dict[question["title"].replace(" ", "").replace("-", "").replace("_", "").lower()] = question["id"]

print(id_title_dict["twosum"])

1


In [15]:
import os


prep_code_dict = {}
test_code_dict = {}

path = "/Users/minkijung/Documents/2PetProjects/ai-mock-coding-interview-agent/backend/db/prep"
for id in os.listdir(path):
    if "test" in id:
        file_path = os.path.join(path, id)
        with open(file_path, "r") as f:
            test_code_dict[id.split("_")[0]] = f.read()

    file_path = os.path.join(path, id)
    with open(file_path, "r") as f:
        data = f.read()
        prep_code_dict[id.split(".")[0]] = data

'import random\nimport string\nimport unittest\n\nfrom hypothesis import given\nfrom hypothesis.strategies import text\n\n\nclass Test(unittest.TestCase):\n    def test_1(self):\n        solution = Solution()\n        self.assertEqual(solution.findTheDifference("abcd", "abcde"), "e")\n\n    @given(text())\n    def test_random(self, s):\n        solution = Solution()\n        random_letter = random.choice(string.ascii_letters)\n        t = list(s + random_letter)\n        random.shuffle(t)\n        t = "".join(t)\n        self.assertEqual(solution.findTheDifference(s, t), random_letter)\n\n\nif __name__ == "__main__":\n    unittest.main()\n'

In [16]:
path = "/Users/minkijung/Documents/2PetProjects/ai-mock-coding-interview-agent/backend/db/leetcode/refined_data"
new_path = "/Users/minkijung/Documents/2PetProjects/ai-mock-coding-interview-agent/backend/db/leetcode/complete_data"

for question in os.listdir(path):
    title = question.split(".")[0].replace(" ", "").replace("-", "").replace("_", "").lower()
    id = id_title_dict.get(title)

    prep_code = prep_code_dict.get(id)
    if prep_code:
        print(f"==>> {title}")
        with open(os.path.join(path, question), "r") as f:
            data = json.load(f)
        data["prep_code"] = prep_code
        data["test_code"] = test_code_dict.get(id)

        if data.get("solution_md"):
            with open(os.path.join(new_path, question), "w") as f:
                json.dump(data, f, indent=4)
        else:
            with open(os.path.join(path, question), "w") as f:
                json.dump(data, f, indent=4)

==>> 3sum
==>> longestpalindromicsubstring
==>> validparentheses
==>> nextpermutation
==>> binarytreeinordertraversal
==>> reverseinteger
==>> containerwithmostwater
==>> mergeintervals
==>> addtwonumbers
==>> longestsubstringwithoutrepeatingcharacters
==>> palindromenumber
==>> longestcommonprefix
==>> mergesortedarray
==>> 4sum
==>> twosum
==>> medianoftwosortedarrays
==>> insertinterval
==>> swapnodesinpairs
==>> lengthoflastword


# Convert to markdown

In [40]:
from markdownify import markdownify
import re
from pprint import pprint

new_path = "/Users/minkijung/Documents/2PetProjects/ai-mock-coding-interview-agent/backend/db/leetcode/complete_data"

for question in os.listdir(new_path):
    with open(os.path.join(new_path, question), "r") as f:
        data = json.load(f)
    data["content_md"] = markdownify(data["content"]).replace("\n\n\u00a0\n\n", "")
    solution = data["solution"]["content"]
    solution = (
        solution.replace('<iframe src="\n', "")
        .replace('"></iframe>', "")
        .replace("", "")
    )
    solution = re.sub(r"frameBorder.*\n", "", solution)
    solution = solution.replace('```\n"', "```\n")

    data["solution_md"] = markdownify(solution)

    approachs = solution.split("\n---\n\n### Approach ")[1:]
    data["approaches"] = []
    for i, approach in enumerate(approachs):
        data["approaches"].append(
            {
                "title": approach.split("\n")[0].replace(f"{i+1}: ", "").strip(),
                "approach": "\n".join(approach.split("\n")[1:]).strip(),
            }
        )
    with open(os.path.join(new_path, question), "w") as f:
        json.dump(data, f, indent=4)

In [2]:
class Solution(object):
    # method to check if a part of the string is within the range 0-255,
    # returns True if part is within range 0-255 else False
    def valid(self, s, start, length):
        return length == 1 or (
            s[start] != "0" and (length < 3 or s[start : start + length] <= "255")
        )

    # main helper method to solve the problem by backtracking
    def helper(self, s, startIndex, dots, ans):
        remainingLength = len(s) - startIndex
        remainingNumberOfIntegers = 4 - len(dots)
        if (
            remainingLength > remainingNumberOfIntegers * 3
            or remainingLength < remainingNumberOfIntegers
        ):
            return
        if len(dots) == 3:
            if self.valid(s, startIndex, remainingLength):
                temp = ""
                last = 0
                for dot in dots:
                    temp += s[last : last + dot] + "."
                    last += dot
                temp += s[startIndex:]
                ans.append(temp)
            return
        for curPos in range(1, min(4, remainingLength + 1)):
            dots.append(curPos)
            if self.valid(s, startIndex, curPos):
                self.helper(s, startIndex + curPos, dots, ans)
            dots.pop()

    # main method called by leetcode
    def restoreIpAddresses(self, s):
        answer = []
        self.helper(s, 0, [], answer)
        return answer

In [9]:
solution = Solution()
result = solution.restoreIpAddresses("12.345.678.901.234.567.890")
print(result)

['255.255.11.135', '255.255.111.35']
